In [667]:
import numpy as np
import pandas as pd
from  sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

In [634]:
DF = pd.read_csv('C:\\Users\\ACER\\OneDrive - Universitas Airlangga\\Kuliah\\Learn-MLZoomCamp2023\\Learn-MLZoomCamp2023\\HomeWork\\raw.githubusercontent.com_alexeygrigorev_mlbookcamp-code_master_chapter-02-car-price_data.csv')
DF.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [635]:
features = ['Make','Model','Year','Engine HP', 'Engine Cylinders','Transmission Type','Vehicle Style','highway MPG','city mpg','MSRP']
df = DF[features]
df

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,MSRP
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


In [636]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [637]:
df.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
msrp                   int64
dtype: object

In [638]:
df = df.fillna(0)
df.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
msrp                 0
dtype: int64

In [639]:
df.rename(columns = {'msrp':'price'}, inplace = True)

In [640]:
df.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'price'],
      dtype='object')

In [641]:
df['transmission_type'].value_counts()

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

In [642]:
df['transmission_type'].mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

In [643]:
numerical = ['engine_hp', 'engine_cylinders','highway_mpg', 'city_mpg']
df[numerical].corr()

,engine_hp,engine_cylinders,highway_mpg,city_mpg
engine_hp,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,-0.415707,-0.614541,1.000000,0.886829
city_mpg,-0.424918,-0.587306,0.886829,1.000000


In [644]:
avg = df['price'].mean()
df['above_average'] = (df['price'] > avg).astype(int)

In [645]:
df1 = df.copy()

In [646]:
x_train,x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train,x_val = train_test_split(x_train,test_size=0.25, random_state=42)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = x_train['above_average'].values
y_val = x_val['above_average'].values
y_test = x_test['above_average'].values

del x_train['above_average']
del x_val['above_average']
del x_test['above_average']

del x_train['price']
del x_val['price']
del x_test['price']

In [647]:
def mi(series):
    return mutual_info_score(series,y_train)

In [648]:
categorical = [x for x in x_train.columns if x  not in numerical]
categorical

['make', 'model', 'year', 'transmission_type', 'vehicle_style']

In [649]:
x_train[categorical].apply(mi).round(2)

make                 0.24
model                0.46
year                 0.07
transmission_type    0.02
vehicle_style        0.08
dtype: float64

In [650]:
dv = DictVectorizer(sparse=False)

train_dicts = x_train[categorical + numerical].to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dict = x_val[categorical + numerical].to_dict(orient='records')
x_val = dv.transform(val_dict)

In [651]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(x_train,y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [652]:
y_pred = model.predict_proba(x_val)[:, 1]
threshold = (y_pred >= 0.5)
ori = (y_val == threshold).mean()
ori.round(2)


0.95

In [653]:

x_train,x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train,x_val = train_test_split(x_train,test_size=0.25, random_state=42)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = x_train['above_average'].values
y_val = x_val['above_average'].values
y_test = x_test['above_average'].values

del x_train['above_average']
del x_val['above_average']
del x_test['above_average']

del x_train['price']
del x_val['price']
del x_test['price']

del x_train['engine_hp']
del x_val['engine_hp']
del x_test['engine_hp']

dv = DictVectorizer(sparse=False)

train_dicts = x_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dict = x_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(x_train,y_train)

y_pred = model.predict_proba(x_val)[:, 1]
threshold = (y_pred >= 0.5)
acc = (y_val == threshold).mean().round(2)
print(ori-acc)


0.025866554762903893


In [654]:

x_train,x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train,x_val = train_test_split(x_train,test_size=0.25, random_state=42)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = x_train['above_average'].values
y_val = x_val['above_average'].values
y_test = x_test['above_average'].values

del x_train['above_average']
del x_val['above_average']
del x_test['above_average']

del x_train['price']
del x_val['price']
del x_test['price']

del x_train['transmission_type']
del x_val['transmission_type']
del x_test['transmission_type']

dv = DictVectorizer(sparse=False)

train_dicts = x_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dict = x_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(x_train,y_train)

y_pred = model.predict_proba(x_val)[:, 1]
threshold = (y_pred >= 0.5)
acc = (y_val == threshold).mean().round(2)
print(ori-acc)


0.005866554762903986


In [655]:

x_train,x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train,x_val = train_test_split(x_train,test_size=0.25, random_state=42)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = x_train['above_average'].values
y_val = x_val['above_average'].values
y_test = x_test['above_average'].values

del x_train['above_average']
del x_val['above_average']
del x_test['above_average']

del x_train['price']
del x_val['price']
del x_test['price']

del x_train['year']
del x_val['year']
del x_test['year']

dv = DictVectorizer(sparse=False)

train_dicts = x_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dict = x_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(x_train,y_train)

y_pred = model.predict_proba(x_val)[:, 1]
threshold = (y_pred >= 0.5)
acc = (y_val == threshold).mean().round(2)
print(ori-acc)


-0.004133445237096023


In [656]:

x_train,x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train,x_val = train_test_split(x_train,test_size=0.25, random_state=42)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = x_train['above_average'].values
y_val = x_val['above_average'].values
y_test = x_test['above_average'].values

del x_train['above_average']
del x_val['above_average']
del x_test['above_average']

del x_train['price']
del x_val['price']
del x_test['price']

del x_train['city_mpg']
del x_val['city_mpg']
del x_test['city_mpg']

dv = DictVectorizer(sparse=False)

train_dicts = x_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dict = x_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(x_train,y_train)

y_pred = model.predict_proba(x_val)[:, 1]
threshold = (y_pred >= 0.5)
acc = (y_val == threshold).mean().round(2)
print(ori-acc)


0.015866554762903884


In [657]:
DF = DF[features]
DF.columns = DF.columns.str.replace(' ', '_').str.lower()
df = DF.fillna(0)
df.rename(columns = {'msrp':'price'}, inplace = True)

In [ ]:

x_train,x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train,x_val = train_test_split(x_train,test_size=0.25, random_state=42)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = np.log1p(x_train['price'].values)
y_val = np.log1p(x_val['price'].values)
y_test = np.log1p(x_test['price'].values)

del x_train['price']
del x_val['price']
del x_test['price']

dv = DictVectorizer(sparse=False)

train_dicts = x_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dict = x_val.to_dict(orient='records')
x_val = dv.transform(val_dict)

alpha = [0, 0.01, 0.1, 1, 10]

for i in alpha :
    model = Ridge(alpha=i,solver='sag',random_state=42)
    model.fit(x_train,y_train)

    y_pred = model.predict(x_val)
    rmse = np.sqrt(((y_val - y_pred)**2).mean())
    print(f"RMSE with Alpha {i} = {rmse}")


